# 音声感情認識モデルの学習とTFLite変換

## 1. 準備とライブラリのインポート

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

print(f'TensorFlow version: {tf.__version__}')

## 2. 定数とパスの定義

In [ ]:
# データセットへの相対パス
DATA_PATH = '../data/'
# 成果物を保存するディレクトリへの相対パス
MODELS_PATH = '../models/'

# モデルファイル名
H5_MODEL_NAME = 'emotion_model.h5'
TFLITE_MODEL_NAME = 'emotion_model.tflite'
LABELS_NAME = 'labels.txt'

# ディレクトリが存在しない場合は作成
if not os.path.exists(MODELS_PATH):
    os.makedirs(MODELS_PATH)
    
print(f'Data path: {os.path.abspath(DATA_PATH)}')
print(f'Models path: {os.path.abspath(MODELS_PATH)}')


## 3. データセットの読み込みと前処理

In [ ]:
# RAVDESS: 01=neutral, 02=calm, 03=happy, 04=sad, 05=angry, 06=fearful, 07=disgust, 08=surprised
emotion_map_ravdess = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprise'
}
# CREMA-D: SAD, ANG, DIS, FEA, HAP, NEU
emotion_map_crema = {
    'SAD': 'sad', 'ANG': 'angry', 'DIS': 'disgust',
    'FEA': 'fearful', 'HAP': 'happy', 'NEU': 'neutral'
}

def extract_feature(file_name):
    """
    ファイルからMFCC特徴量を抽出する.
    READMEの要件に基づき、40次元のMFCCを平均化する.
    """
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast', duration=3, sr=16000)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"特徴量抽出エラー: {file_name} - {e}")
        return None
    return mfccs_processed

all_features = []
all_labels = []

print("データセットをスキャンしています...")
if not os.path.exists(DATA_PATH) or not os.listdir(DATA_PATH):
    print("!!!警告!!!")
    print(f"データディレクトリ '{os.path.abspath(DATA_PATH)}' が空か、存在しません。")
    print("RAVDESS と CREMA-D データセットをこのディレクトリに配置してください。")
else:
    for root, dirs, files in os.walk(DATA_PATH):
        for file in files:
            if file.endswith('.wav'):
                emotion = None
                file_path = os.path.join(root, file)
                
                # ファイル名から感情を判定 (RAVDESS)
                parts = file.split('.')[0].split('-')
                if len(parts) == 7 and parts[2] in emotion_map_ravdess:
                    emotion = emotion_map_ravdess[parts[2]]
                
                # ファイル名から感情を判定 (CREMA-D)
                else:
                    parts = file.split('.')[0].split('_')
                    if len(parts) >= 3 and parts[2] in emotion_map_crema:
                        emotion = emotion_map_crema[parts[2]]
                
                if emotion:
                    feature = extract_feature(file_path)
                    if feature is not None:
                        all_features.append(feature)
                        all_labels.append(emotion)
    
    if not all_features:
        print("!!!警告!!!")
        print(f"データディレクトリ '{os.path.abspath(DATA_PATH)}' から音声ファイルが見つかりませんでした。")
        print("RAVDESS と CREMA-D データセットをこのディレクトリに配置してください。")
        print("例: ../data/RAVDESS/Actor_01/03-01-01-01-01-01-01.wav")
    else:
        print(f"特徴量抽出完了。合計: {len(all_features)} ファイル。")
        print(f"ユニークなラベル: {sorted(np.unique(all_labels))}")

### 3.1 データの整形と分割

In [ ]:
if all_features:
    # 特徴量とラベルをNumpy配列に変換
    X = np.array(all_features)
    y = np.array(all_labels)
    
    # ラベルエンコーダ
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded)
    
    # ラベルのマッピングを表示
    print("感情ラベルのマッピング:")
    classes = sorted(label_encoder.classes_)
    for i, label in enumerate(classes):
        print(f"{i}: {label}")
        
    # 学習データとテストデータに分割 (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, stratify=y_categorical)
    
    # データをCNNの入力形式に合わせる (features, 1)
    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)
    
    print(f"\n学習データ数: {len(X_train)}")
    print(f"テストデータ数: {len(X_test)}")
    print(f"入力データの形状: {X_train.shape}")
    print(f"ラベルデータの形状: {y_train.shape}")
    
    # ラベルクラスをファイルに保存
    with open(os.path.join(MODELS_PATH, LABELS_NAME), 'w') as f:
        for label in classes:
            f.write(f'{label}\n')
    print(f'\nラベルを {os.path.join(MODELS_PATH, LABELS_NAME)} に保存しました。')

## 4. モデルの定義 (1D-CNN)

In [ ]:
if all_features:
    input_shape = (X_train.shape[1], 1)
    num_classes = y_train.shape[1]

    model = Sequential([
        Conv1D(128, 5, padding='same', activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        Conv1D(256, 5, padding='same', activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.summary()

## 5. モデルの学習

In [ ]:
if all_features:
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # 学習の実行
    epochs = 50
    batch_size = 32
    
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        verbose=1
    )
    
    # 学習済みモデルを保存
    model.save(os.path.join(MODELS_PATH, H5_MODEL_NAME))
    print(f"\nモデルを {os.path.join(MODELS_PATH, H5_MODEL_NAME)} に保存しました。")

## 6. モデルの評価と可視化

In [ ]:
if all_features:
    # 評価
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nテストデータでの精度: {accuracy:.4f}")
    print(f"テストデータでの損失: {loss:.4f}")

    # 可視化
    plt.figure(figsize=(12, 4))

    # 精度のプロット
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # 損失のプロット
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

## 7. TFLiteへの変換と量子化

In [ ]:
if all_features:
    # 保存したKerasモデルをロード
    keras_model_path = os.path.join(MODELS_PATH, H5_MODEL_NAME)
    model = load_model(keras_model_path)

    # TFLiteコンバータを作成
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    # 量子化を有効にする (デフォルト最適化)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    
    # モデルを変換
    tflite_model = converter.convert()
    
    # TFLiteモデルを保存
    tflite_model_path = os.path.join(MODELS_PATH, TFLITE_MODEL_NAME)
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
        
    print(f"TFLiteモデルを {tflite_model_path} に保存しました。")
    
    # ファイルサイズを比較
    keras_model_size = os.path.getsize(keras_model_path) / 1024
    tflite_model_size = os.path.getsize(tflite_model_path) / 1024
    
    print(f"\n元のKerasモデルサイズ: {keras_model_size:.2f} KB")
    print(f"変換後のTFLiteモデルサイズ: {tflite_model_size:.2f} KB")
    print(f"削減率: {(1 - tflite_model_size / keras_model_size) * 100:.2f}%")

## 8. TFLiteモデルのテスト

In [ ]:
if all_features:
    # TFLiteモデルをロード
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    
    # 入出力テンソルの詳細を取得
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # テストデータで推論を実行
    # 最初のテストデータを取得
    input_data = np.expand_dims(X_test[0], axis=0).astype(np.float32)
    
    # データを入力テンソルにセット
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # 推論を実行
    interpreter.invoke()
    
    # 出力テンソルから結果を取得
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    predicted_label_index = np.argmax(output_data)
    predicted_label = classes[predicted_label_index]
    
    true_label_index = np.argmax(y_test[0])
    true_label = classes[true_label_index]
    
    print("TFLiteモデルでの推論テスト:")
    print(f"予測: {predicted_label} (index: {predicted_label_index})")
    print(f"正解: {true_label} (index: {true_label_index})")
    print(f"出力確率: {output_data[0]}")